In [19]:
import pandas as pd
import re

In [20]:
def process(filename):
    df = pd.read_excel(filename,sheet_name="Sheet1")
    temp1 = df.iloc[:,1]  
    temp2 = df.iloc[:,3]  
    temp3 = df.iloc[:,6]
    pattern1 = re.compile('(.[\u4E00-\u9FA5]+)')  
    pattern2 = re.compile('([^\u4E00-\u9FA5]+)')  
    df["航空公司"] = temp1.str.extract(pattern1)  
    df["航班号"] = temp1.str.extract(pattern2)    
    df["起飞机场"] = temp2.str.extract(pattern1)  
    df["达到机场"] = temp3.str.extract(pattern1)  
    newfilename = filename.split(".")[0] + ".csv"
    df.to_csv(newfilename,sep=',',index=False)    

In [21]:
process("flights_1_1000.xlsx") 

In [27]:
def t_process(filename):
    df = pd.read_excel(filename,sheet_name="Sheet1")
    keys = pd.read_excel("airportChina V1.1.xlsx",sheet_name="Sheet1")
    keys = keys[["机场名称","机场三字码","机场四字码"]]  #获取这三列
    names = keys["机场名称"].tolist()#机场名
    code_3 = keys["机场三字码"].tolist()#3字码
    code_4 = keys["机场四字码"].tolist()#4字码
    dict1 = dict(zip(names,code_3))#机场名-3字码
    dict2 = dict(zip(names,code_4))#机场名-4字码
    temp1 = df.iloc[:,1] #获取第1列
    temp2 = df.iloc[:,3]
    temp3 = df.iloc[:,6]
    pattern1 = re.compile('(.[\u4E00-\u9FA5]+)')  #创建模式对象，匹配所有中文字符
    pattern2 = re.compile('([^\u4E00-\u9FA5]+)')  #匹配除中文字符以外的所有字符
    df["航空公司"] = temp1.str.extract(pattern1)  #获取这一匹配
    df["航班号"] = temp1.str.extract(pattern2)
    df["起飞机场"] = temp2.str.extract(pattern1)
    df["达到机场"] = temp3.str.extract(pattern1)
    departure = df["起飞机场"]
    arrival = df["达到机场"]
    #arrival中的元素不在机场名称里的行数和departure中的元素不在机场名称里的行数和，并将其打印
    test = arrival[~arrival.isin(keys["机场名称"])].shape[0] + departure[~departure.isin(keys["机场名称"])].shape[0] 
    if (test > 0):
        print("There exists "+str(test)+" unmatched airport names.")
        print("For the arrival part:")
        print(df[~arrival.isin(keys["机场名称"])])
        print("For the departure part:")
        print(df[~departure.isin(keys["机场名称"])])
    df["3字码起飞"] = departure.replace(dict1)  #将起飞机场名用3字码代替
    df["4字码起飞"] = departure.replace(dict2)  #将起飞机场名用4字码代替
    df["3字码降落"] = arrival.replace(dict1)
    df["4字码降落"] = arrival.replace(dict2)
    newfilename = filename.split(".")[0] + ".csv"
    df.to_csv(newfilename,sep=',',index=False)
    return pd.DataFrame(df)

In [28]:
df2 = t_process("flights_1_1000.xlsx")